In [16]:
import pandas as pd
import os
import shutil

In [17]:
imageDir = '../Images/'
csvDir = '../CSV'
basePath = '../MTLData/'
splitCsv = 'csv'

In [18]:
def dataLoader(imageDir,csvFile,sampleRatio=0.60):
    
    trainSplitFactor = sampleRatio
    testSplitFactor = valdSplitFactor = (1-sampleRatio)/2
    
    train = pd.DataFrame()
    test = pd.DataFrame()
    vald = pd.DataFrame()
    data = pd.DataFrame()
    
    csv = os.path.join(csvDir,csvFile)
    if os.path.isfile(csv) == False:
        exit(1)
    if os.path.isdir(imageDir) == False:
        exit(1)
    
    csvData = pd.read_csv(csv)
    dataShuffled = csvData.sample(frac=1).reset_index(drop=True) #Shuffle data

    print('\n\nProcessing files')
    total = dataShuffled.shape[0]
    trainLen = int(total*trainSplitFactor)
    valdLen = int(total*valdSplitFactor)
    testLen = int(total*testSplitFactor)

    print("\tTotal: {ttl}, Train: {tr}, Test: {te}, Vald: {vld}".format(ttl=total,tr=trainLen,te=testLen,vld = valdLen))
    trainData = dataShuffled[0:trainLen]
    valdData = dataShuffled[trainLen+1: (trainLen+valdLen)]
    testData = dataShuffled[(trainLen+valdLen)+1:(trainLen+valdLen+testLen)]

    imagesTrain = trainData['Image'].tolist()
    imagesVald = valdData['Image'].tolist()
    imagesTest = testData['Image'].tolist()
    
    splits = ['train','test','vald']
    cpCount = 0
    for sp in splits:
        if sp == 'train':
            images = imagesTrain
            tLen = len(imagesTrain)
            train = trainData
            train['dir'] = 'train'
            data = pd.concat([data,train])
        elif sp == 'test':
            images = imagesTest
            test = testData
            test['dir'] = 'test'
            data = pd.concat([data,test])
            tLen = len(imagesTest)
        elif sp == 'vald':
            images = imagesVald
            vald = valdData
            vald['dir'] = 'vald'
            data = pd.concat([data,vald])
            tLen = len(imagesVald)
            
        for image in images:
            copyFrom = os.path.join(imageDir,image+'.jpg')
            copyTo = os.path.join(basePath,sp+'/',image+'.jpg')

            nestedDir = os.path.join(basePath,sp+'/')
            if not os.path.exists(nestedDir):
                os.makedirs(nestedDir)

            shutil.copy(copyFrom, copyTo)
            print('\t\tCopied {cp} of {ttl}'.format(cp=cpCount,ttl=tLen),end="\r")
            cpCount += 1

    CSVPath = os.path.join(basePath,splitCsv)
    if not os.path.exists(CSVPath+'/'):
        os.makedirs(CSVPath+'/')
    data.to_csv(os.path.join(CSVPath+'/','data.csv'))
    print('Data Saved to: ',os.path.join(CSVPath+'/','data.csv'))

In [19]:
dataLoader(imageDir,'cleanedData.csv',sampleRatio=0.70)



Processing files
	Total: 10699, Train: 7489, Test: 1604, Vald: 1604


C:\Users\z0045jzp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


		Copied 7513 of 1603

C:\Users\z0045jzp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\z0045jzp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Data Saved to:  ../MTLData/csv/data.csv


In [20]:
d = pd.read_csv('../MTLData/csv/data.csv')

In [21]:
fuel = d.groupby(['PFRType','FuelType','NOx']).size().reset_index().rename(columns={0:'count'})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(fuel)

   PFRType  FuelType    NOx  count
0        0  20H280NG    0-5    864
1        0  20H280NG  06-10    176
2        0   Ethlyne    0-5   1626
3        0        F1    0-5    244
4        0        F1  06-10     95
5        0        F2    0-5    420
6        0        F2  06-10    197
7        0        NG    0-5   2151
8      1-3  20H280NG    0-5     77
9      1-3   Ethlyne    0-5     96
10     1-3        F2    0-5     71
11     1-3        F2  06-10     78
12     1-3        NG    0-5    196
13   10-12  20H280NG    0-5     34
14   10-12  20H280NG  06-10    190
15   10-12   Ethlyne  06-10    122
16   10-12        F1  06-10     34
17   10-12        F2  06-10     75
18   10-12        NG    0-5    555
19   10-12        NG  06-10     56
20   13-15  20H280NG  06-10     79
21   13-15        F2  11-15     53
22   13-15        NG    0-5    109
23   13-15        NG  06-10     63
24   20-25  20H280NG  06-10     30
25   20-25  20H280NG  11-15     35
26   20-25        F1  06-10      2
27   20-25        NG

In [22]:
 
indexNames = d[ (d['FuelType'] == 'NG') & (d['NOx'] == '16-20') ].index
d.drop(indexNames , inplace=True)

indexNames = d[ (d['FuelType'] == 'F1') & (d['NOx'] == '06-10') ].index
d.drop(indexNames , inplace=True)

In [23]:
fuel = d.groupby(['PFRType','FuelType','NOx']).size().reset_index().rename(columns={0:'count'})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(fuel)

   PFRType  FuelType    NOx  count
0        0  20H280NG    0-5    864
1        0  20H280NG  06-10    176
2        0   Ethlyne    0-5   1626
3        0        F1    0-5    244
4        0        F2    0-5    420
5        0        F2  06-10    197
6        0        NG    0-5   2151
7      1-3  20H280NG    0-5     77
8      1-3   Ethlyne    0-5     96
9      1-3        F2    0-5     71
10     1-3        F2  06-10     78
11     1-3        NG    0-5    196
12   10-12  20H280NG    0-5     34
13   10-12  20H280NG  06-10    190
14   10-12   Ethlyne  06-10    122
15   10-12        F2  06-10     75
16   10-12        NG    0-5    555
17   10-12        NG  06-10     56
18   13-15  20H280NG  06-10     79
19   13-15        F2  11-15     53
20   13-15        NG    0-5    109
21   13-15        NG  06-10     63
22   20-25  20H280NG  06-10     30
23   20-25  20H280NG  11-15     35
24   20-25        NG    0-5     74
25   20-25        NG  06-10    289
26   30-40  20H280NG  11-15     67
27   30-40  20H280NG

In [24]:
d.to_csv('../MTLData/csv/data.csv')